In [1]:
import pandas as pd
from openpyxl import load_workbook
import openpyxl
import os
import glob
from datetime import datetime
import matplotlib.pyplot as plt
import numpy as np

In [2]:
path1 = r"C:\Computer_Science\WS325705\Marwan Albanna\Desktop -"

In [3]:
preheat_path = [files for files in glob.glob(path1+"\Room A\Preheat File Temporary\\" + '*')]

In [4]:
information_path = path1+"\Cell Comparisons JA vs AT vs Techno.xlsx"
information_data = pd.read_excel(information_path,'Raw')
information_data = information_data.iloc[:36,:]

In [5]:
def get_index(x,data):
    index = -1
    for i in range(1,len(data)):
        if data.iloc[i,9]>x and data.iloc[i-1,9]<x:
            index = i-1
            break
    return index

def get_list(index,col,data):   
    ll = []
    for i in range(len(data)):
        if i == 0:
            ll.append(-1)
        elif i == index:
            ll.append(0)
        else:
            if int(data.iloc[i,22].timestamp()) & int(data.iloc[i-1,22].timestamp()):
                if ll[i-1]>0:
                    ll.append(ll[i-1] + 1)
                else:
                    if data.iloc[i,col]>2 and data.iloc[i-1,col]<1:
                        ll.append(1)
                    else:
                        ll.append(-1)
            else:
                ll.append(-1)
    return ll

In [6]:
for php in preheat_path:
    print(int(php.split('\\')[-1].split('.')[0].split('-')[0][2:]))
    xls_number = int(php.split('\\')[-1].split('.')[0].split('-')[0][2:])
#     if xls_number == 8:
#         break
    path = path1+"\\"+str(xls_number)+".xlsx"
    print(path)
    writer = pd.ExcelWriter(path,engine='openpyxl')
    
    # Life
    life_path = path1+"\\Room A\\Life Temporary"+"\\"+str(xls_number)+".csv"
    print("Life Path",life_path)
    life_data = pd.read_csv(life_path)
    life_data['Date'] = pd.to_datetime(life_data['CT_DATE'])
    life_data = life_data.sort_values(by='Date').reset_index().drop(['index'],1)
    
    cut_in_index = get_index(2,life_data)
    time_cut_in = get_list(cut_in_index,9,life_data)
    cutin_date = life_data.loc[cut_in_index,'Date']
    
    life_data['Time From Cut-in'] = time_cut_in
    life_data['Time from Bath-Up'] = ''
    
    life_data['CRSP_V'] = life_data['CT_CRSP']/1000*life_data['CT_RAW_CURRENT']+1.65
    life_data['RAW_RESIS'] = (life_data['CT_RAW_VOLT']+1.65)/life_data['CT_RAW_CURRENT']
    life_data['NOISE_V'] = life_data['CT_NOISE']/1000
    life_data['Power kW'] = life_data['CT_RAW_VOLT']*life_data['CT_RAW_CURRENT']*1/60
    life_data['Just a #'] = 1000
    life_data['Power MW'] = life_data['Power kW'] / life_data['Just a #']
    
    pow_mw_hr = []
    for i in range(len(life_data)):
        if i == 0:
            pow_mw_hr.append(life_data.loc[0,'Power MW'])
        else:
            pow_mw_hr.append(pow_mw_hr[i-1] + life_data.loc[i,'Power MW'])
            
    life_data['Power MW/HR'] = pow_mw_hr
    
    life_data = life_data[['Date','Time From Cut-in','Time from Bath-Up','CT_DATE', 'CELL_START_DATE', 'CT_CELL_ID', 'CT_BEAM_POS',
       'CT_FEED_RATE', 'CT_SM_RES', 'CT_BRSP', 'CT_CRSP', 'CT_NOISE',
       'CT_RAW_VOLT', 'CT_RAW_CURRENT', 'CD_OFFLINE', 'CD_CATH_ID',
       'CD_POT_AGE', 'CD_REST_POT_AGE', 'CD_PURITY', 'CT_FEED_WIN', 'CD_CE',
       'CD_LATEST_BATH_TEMP', 'CD_ALF3_ADDED', 'CD_NA2CO3_ADDED',
       'CD_CAF2_ADDED','CRSP_V', 'RAW_RESIS', 'NOISE_V', 'Power kW', 'Just a #', 'Power MW',
       'Power MW/HR']]
    
    if len(life_data) > 1048575:
        life_data = life_data.iloc[:1048575,:]
    
    life_data.to_excel(writer, sheet_name="Life",index = False)
    print("Done with  Life Data")
    
    #Preheat
    print("Preheat Path : ",php)
    preheat_data = pd.read_excel(php,"Preheat Trend", usecols="B:L",header=108)
    for i in range(len(preheat_data)):
        try:
            if np.isnan(preheat_data.iloc[i,0]):
                index = i
                break
        except:
            continue
    preheat_data = preheat_data.iloc[:index,:]
    preheat_data['Date - Time'] = pd.to_datetime(preheat_data['Date - Time']).astype('datetime64[s]')
    preheat_data = preheat_data[['Date - Time','Duration','27-C\nBK 1','21-C\nBK 2','14-C\nBK 3','8-C\nBK 4','2-C\nBK 5','Average']]
    
    data_raw = pd.read_excel(php,"Calc",header=1)[['Date','Cum.Energy.1']]
    data_raw['Date - Time'] = pd.to_datetime(data_raw['Date']).astype('datetime64[s]')
    new_data  = pd.merge(data_raw, preheat_data, on ='Date - Time', how ='right')
    new_data['PH_ENERGY'] = new_data['Cum.Energy.1']
    new_data['Time'] = new_data['Duration']
    new_data = new_data[['Date - Time','Time','PH_ENERGY','27-C\nBK 1','21-C\nBK 2','14-C\nBK 3','8-C\nBK 4','2-C\nBK 5','Average']]
    new_data['Watt/HR to Celsius'] = new_data['Average'] * 1.9
    new_data['Watt/HR to Energy/HR'] = new_data['Average'] * 3600
    new_data['Time From Cut-in'] = ((new_data['Date - Time']-cutin_date)*60*24).dt.days
    
    new_data['Time Before the Bath-up'] = ''
    
    new_data = new_data[['Date - Time','Time','PH_ENERGY','27-C\nBK 1','21-C\nBK 2','14-C\nBK 3','8-C\nBK 4','2-C\nBK 5','Time From Cut-in','Time Before the Bath-up','Average','Watt/HR to Celsius','Watt/HR to Energy/HR']]
    new_data.to_excel(writer, sheet_name="PreHeat",index = False)
    
    #Information
    infor_index = -1
    for j32 in range(len(information_data)):
        if int(information_data.loc[j32,'Cell'].split('A')[1]) == xls_number:
            print(information_data.loc[j32,'Cell'])
            infor_index = j32
    
    #information
    x = information_data.iloc[infor_index,1:-4]
    Name = list(x.index)
    Value =  list(x.values)
    Name.insert(0,'Cut-in Time')
    Name.insert(2,'Bath-up Time')
    Value.insert(0,cutin_date)
    Value.insert(2,'')
    frame = {'name':Name,'value':Value}
    frame = pd.DataFrame(frame) 
    frame = frame.drop(labels=[3,4,18,23], axis=0).reset_index().drop(['index'],1)
    frame.iloc[12,0] = 'Total Preheat Duration'
    frame.iloc[13,0] = 'Total Given Energy'
    frame.iloc[14,0] = 'Cathode Temperature (T/E)'
    frame.iloc[15,0] = 'Cathode Temperature (D/E)'
    frame.iloc[17,0] = 'Preheat Mean Volt'
    frame.iloc[18,0] = 'Preheat Mean Current'
    frame.iloc[21,0] = 'PAH Model'
    b, c = frame.iloc[20], frame.iloc[21]
    temp = frame.iloc[20].copy()
    frame.iloc[20] = c
    frame.iloc[21] = temp
    frame.to_excel(writer, sheet_name="Informations",index = False)
    
    
    
    #Action
    action_path = path1+"\Room A\Action File Temporary"+"\\"+str(xls_number)+".csv"
    print(action_path)
    action_data = pd.read_csv(action_path)
    action_data['Date'] = pd.to_datetime(action_data['CT_DATE'])
    action_data = action_data.sort_values(by='Date').reset_index().drop(['index'],1)
    action_data['Time From Cut-in'] = ((action_data['Date']-cutin_date)*60*24).dt.days
    action_data['Time From Bath-up'] = ''
    action_data['Total Liquid Before Tap'] = action_data['CD_BATH_DEPTH_BF_TAP']+ action_data['CD_METAL_DEPTH_BF_TAP']
    action_data['Total Liquid After Tap'] = action_data['CD_BATH_DEPTH_AF_TAP']+ action_data['CD_METAL_DEPTH_AF_TAP']
    action_data['Total Liquid Before Tap']  = action_data['Total Liquid Before Tap'].fillna(0)
    action_data['Total Liquid After Tap']  = action_data['Total Liquid After Tap'].fillna(0)
    
    
    action_data = action_data[['Date','Time From Cut-in','Time From Bath-up','CT_DATE', 'CELL_START_DATE', 'CT_CELL_ID', 'CD_OFFLINE', 'CD_POT_AGE',
       'CD_REST_POT_AGE', 'CD_BATH_TEMP', 'CD_BATH_DEPTH_BF_TAP',
       'CD_BATH_DEPTH_AF_TAP', 'CD_METAL_DEPTH_BF_TAP',
       'CD_METAL_DEPTH_AF_TAP', 'CD_ALF3_ADDED', 'CD_NA2CO3_ADDED',
       'CD_CAF2_ADDED', 'CD_XS_ALF3', 'CD_SPAR', 'CD_CRYOLITE_ADDED',
       'CD_LIQUID_BATH_ADDED', 'CD_METAL_ADDED', 'CD_LIQUID_METAL_ADDED',
       'CD_POTLINE_TAP_WT', 'CD_NO_AE','Total Liquid Before Tap','Total Liquid After Tap']]
    
    if len(action_data) > 1048575:
        action_data = action_data.iloc[:1048575,:]
        
    action_data.to_excel(writer, sheet_name="Action List",index = False)
    print("Done")
    writer.save()
    

6
E:\Computer_Science\internshipp\Nitesh Kumar\Project 70 -\6.xlsx
Life Path E:\Computer_Science\internshipp\Nitesh Kumar\Project 70 -\Room A\Life Temporary\6.csv


C:\Users\deep\AppData\Local\Temp/ipykernel_7192/861896857.py:15: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  life_data = life_data.sort_values(by='Date').reset_index().drop(['index'],1)


Done with  Life Data
Preheat Path :  E:\Computer_Science\internshipp\Nitesh Kumar\Project 70 -\Room A\Preheat File Temporary\1A006-Preheat Energy Trend.xlsx
1A006
E:\Computer_Science\internshipp\Nitesh Kumar\Project 70 -\Room A\Action File Temporary\6.csv


C:\Users\deep\AppData\Local\Temp/ipykernel_7192/861896857.py:100: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  frame = frame.drop(labels=[3,4,18,23], axis=0).reset_index().drop(['index'],1)
C:\Users\deep\AppData\Local\Temp/ipykernel_7192/861896857.py:121: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  action_data = action_data.sort_values(by='Date').reset_index().drop(['index'],1)


Done
8
